In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler
import scipy.sparse as sparse
import implicit

In [2]:
df2 = pd.read_csv("./data/processed_data.csv", encoding="utf-8")

In [3]:
df2.head()

,user_id,order_id,time,description,product_id,product_name,amount,price
0,BDS49BDS,XBH011609/006,2016-09-05,Xuất bán T09/06- DYKEH ngày ETD 5/9/2016,LTSCORMH374SCA1,Corner Sofa Set_Middle_H374,68.0,1812.88
1,BDS49BDS,XBH011609/005,2016-09-05,Xuất bán T09/05- DYKEH ngày ETD 5/9/2016,LTSCORMH374SCA1,Corner Sofa Set_Middle_H374,68.0,1812.88
2,BDS49BDS,XBH011609/005,2016-09-05,Xuất bán T09/05- DYKEH ngày ETD 5/9/2016,LTSCORCH374SCA1,Corner Sofa Set_Corner_H374,34.0,1813.22
3,BDS49BDS,XBH011609/005,2016-09-05,Xuất bán T09/05- DYKEH ngày ETD 5/9/2016,LTSCOR1H374SCA1,Corner Sofa Set_ArmChair_H374,34.0,1813.22
4,BDS49BDS,XBH011609/004,2016-09-05,Xuất bán T09/04- DYKEH ngày ETD 5/9/2016,LTTCORVH374SCA1,Corner Sofa Set_Table_H374,34.0,1814.24


In [4]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2415 entries, 0 to 2414
Data columns (total 8 columns):
user_id         2415 non-null object
order_id        2415 non-null object
time            2415 non-null object
description     2415 non-null object
product_id      2415 non-null object
product_name    2415 non-null object
amount          2415 non-null float64
price           2415 non-null float64
dtypes: float64(2), object(6)
memory usage: 151.1+ KB


In [5]:
df2.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
user_id,2415,45,AQU49LCW,422,NaN,NaN,NaN,NaN,NaN,NaN,NaN
order_id,2415,1132,XBH011606/026,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
time,2415,224,2017-01-21,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,2415,1131,Xuất bán T06/26 - LT9516CLFA ngày ETD 26/6/2016,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_id,2415,496,LTCGUD1F310SPA1,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_name,2415,495,Gudhjem Chair_F310,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
amount,2415,NaN,NaN,NaN,382.737,567.262,1,14,78,720,5040
price,2415,NaN,NaN,NaN,5540.1,6231.51,0,637.2,2618,10080,31620


### Summary
- user
    - Total: 45
    - Most freq buyer: AQU49LCW (422 times)
    - 
- product
    - Total: 496
    - Most popular product: LTCGUD1F310SPA1 (86 times)

In [6]:
df2["time"] = df2["time"].astype(np.datetime64)
df2["user_id"] = df2["user_id"].astype("category")
df2["product_id"] = df2["product_id"].astype("category")
df2["is_train"] = 1
df2["username"] = df2["user_id"]
df2["user_id"] = df2["user_id"].astype("category").cat.codes
df2["product_code"] = df2["product_id"]
df2["product_id"] = df2["product_id"].astype("category").cat.codes
df2.sort_values(by=['time'], inplace=True)
df2.reset_index(drop=True,inplace=True)
df2

,user_id,order_id,time,description,product_id,product_name,amount,price,is_train,username,product_code
0,37,XBH011605/001,2016-05-04,Xuất bán T05/01- 266-G ngày ETD 07/05/2016,94,King Chair_H224,122.0,8418.0,1,SSI31SSI,LTCKIN1H224AC1
1,21,XBH011605/002,2016-05-05,Xuất bán T05/02-LTINV0213 ngày ETD 9/5/2016,73,Gudhjem Chair_F210,1512.0,16480.8,1,LIK45JYS,LTCGUD1F210SPA1
2,8,XHM011605/001,2016-05-06,Xuất bán hàng mẫu -SAMPLE 06052016,232,Mẫu Sectional roma set (K/D)_LSX16031,1.0,179.0,1,BEM41BEM,LTSAM00093WI
3,21,XBH011605/007,2016-05-06,Xuất bán T05/07- LTINV6740 ngày ETD 12/5/2016,73,Gudhjem Chair_F210,1512.0,16480.8,1,LIK45JYS,LTCGUD1F210SPA1
4,21,XBH011605/006,2016-05-06,Xuất bán T05/06- LTINV7506 ngày ETD 08/5/2016,73,Gudhjem Chair_F210,1512.0,16480.8,1,LIK45JYS,LTCGUD1F210SPA1
...,...,...,...,...,...,...,...,...,...,...,...
2410,20,XBH011704/141,2017-04-28,Xuất bán T04/141 - 26982/2131201 - LSX 73,494,Nút nhựa f27.6 (bali),5040.0,0.0,1,LIK45DAV,VTNUT027002
2411,20,XBH011704/141,2017-04-28,Xuất bán T04/141 - 26982/2131201 - LSX 73,138,Nệm ngồi Bali,720.0,1224.0,1,LIK45DAV,LTCS10081
2412,21,XBH011704/143,2017-04-28,Xuất bán T04/143 - LTINV3100 - LSX 35,38,New Bork Chair F310,720.0,10512.0,1,LIK45JYS,LTCBOR1F310SK1
2413,21,XBH011704/140,2017-04-28,Xuất bán T04/140 - LTINV7042 - LSX 35,26,New Birkemose Chair_F310,1976.0,20056.4,1,LIK45JYS,LTCBIR2F310SP1


In [7]:
product_df = df2.drop_duplicates(subset=["product_id"])[["product_id", "product_code", "product_name", "price", "description"]]

In [8]:
df2[df2.product_id == 4]

,user_id,order_id,time,description,product_id,product_name,amount,price,is_train,username,product_code
1198,2,XBH011612/115,2016-12-31,Xuất bán T12/115 - RUTLUBLUBLJANA1 - LSX 42,4,Albi Bench_F310,97.0,3880.0,1,AQU49LCW,LTBALB1F310SCK1
1574,2,XBH011701/094,2017-01-21,Xuất bán T01/094 - HEINSTAD -STADTHAGEN2 - LS...,4,Albi Bench_F310,21.0,840.0,1,AQU49LCW,LTBALB1F310SCK1
1683,2,XBH011701/129,2017-01-24,"Xuất bán T01/129 - HEINSTAD, BRAFREI, OSTWIT1 ...",4,Albi Bench_F310,2.0,80.0,1,AQU49LCW,LTBALB1F310SCK1


In [9]:
product_df = df2.groupby(["product_id", "product_code", "product_name", "description"]).apply(lambda x: x["price"]/x["amount"]).reset_index().drop(columns=["level_4"], axis=1)
product_df = product_df.drop_duplicates(subset=["product_id", "product_code", "product_name"]).reset_index(drop=True)
product_df.to_csv("./data/product_db.csv", header=False, index=False, sep=";")

In [10]:
product_df = pd.read_csv("./data/product_db.csv", sep=";", header=None)
product_df

,0,1,2,3,4
0,0,LIC0001F000SAC1,Ghế Tồn Kho,Xuất bán T10/013 - 07102016 - Hàng kho Showroom,28.4
1,1,LIS0001F000SAC1,Set Hàng Tồn Kho,Xuất bán T10/013 - 07102016 - Hàng kho Showroom,266.0
2,2,LISREX1F332SCA2,RELAX SUNLOUNGER__F332,Xuất bán T10/019 - 15062016 - LSX 48,52.5
3,3,LISUNL1F000SAC1,Giường Tồn Kho,Xuất bán T10/013 - 07102016 - Hàng kho Showroom,109.8
4,4,LTBALB1F310SCK1,Albi Bench_F310,Xuất bán T01/094 - HEINSTAD -STADTHAGEN2 - LS...,40.0
...,...,...,...,...,...
491,491,VTGLR111001,Kính trong 360x1110x5mm,Xuất mẫu đi test lần 2 theo phiếu yêu cầu xuất...,0.0
492,492,VTGLS049001,Kính cường lực 5ly Ø490x5,Xuất tặng (khách hàng OBI) Chị Thanh - theo ph...,0.0
493,493,VTNUR022001,Nút Đế Nhựa f22,Xuất bán T03/151 - LTINV3890 - LSX 54,0.0
494,494,VTNUT027002,Nút nhựa f27.6 (bali),Xuất bán T04/141 - 26982/2131201 - LSX 73,0.0


In [11]:
user_df = df2[["user_id", "username"]].drop_duplicates().sort_values(["user_id"]).reset_index(drop=True)
user_df.to_csv("./data/user_db.csv", header=False, index=False, sep=";")

In [12]:
user_df = pd.read_csv("./data/user_db.csv", sep=";", header=None)
user_df.head()

,0,1
0,0,AND45HAN
1,1,AQU49GAR
2,2,AQU49LCW
3,3,AQU51FAL
4,4,AQU56FAL


In [13]:
df2[["time", "user_id", "product_id"]].sort_values("time").to_csv("./data/event_db.csv", header=False, index=False, sep=";")

In [14]:
def train_test_split(sample_df):
    test_idx = int((1-0.3)*len(sample_df))
    if test_idx > 0:
        sample_df.is_train.iloc[test_idx:] = 0
    return sample_df
train_test_split_df = df2.groupby(["user_id"]).apply(train_test_split)
train_test_split_df.to_csv("./data/company_train_test_splitted_data_new_041219.csv", index=False)

In [15]:
train_df = train_test_split_df[train_test_split_df.is_train == 1]
test_df = train_test_split_df[train_test_split_df.is_train == 0]

In [16]:
train_df.describe(include="all").T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
user_id,1669,NaN,NaN,NaN,NaT,NaT,17.6195,11.8992,0,4,21,25,44
order_id,1669,810,XBH011606/026,35,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
time,1669,191,2016-12-31 00:00:00,82,2016-05-04,2017-04-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,1669,809,Xuất bán T06/26 - LT9516CLFA ngày ETD 26/6/2016,35,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_id,1669,NaN,NaN,NaN,NaT,NaT,188.212,151.958,0,52,129,355,491
product_name,1669,392,Gudhjem Chair_F310,56,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
amount,1669,NaN,NaN,NaN,NaT,NaT,392.724,567.394,1,14,80,720,1976
price,1669,NaN,NaN,NaN,NaT,NaT,5663.58,6281.38,0,652,2618,10080,31312
is_train,1669,NaN,NaN,NaN,NaT,NaT,1,0,1,1,1,1,1
username,1669,45,AQU49LCW,295,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
test_df.describe(include="all").T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
user_id,746,NaN,NaN,NaN,NaT,NaT,17.8338,12.0145,0,4,21,26.75,44
order_id,746,362,XBH011701/129,21,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
time,746,100,2017-01-21 00:00:00,57,2016-05-06,2017-04-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,746,362,"Xuất bán T01/129 - HEINSTAD, BRAFREI, OSTWIT1 ...",21,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_id,746,NaN,NaN,NaN,NaT,NaT,201.193,158.424,2,56.25,144.5,364.75,495
product_name,746,292,New Bork Chair_F314,31,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
amount,746,NaN,NaN,NaN,NaT,NaT,360.394,566.712,1,12.25,60,720,5040
price,746,NaN,NaN,NaN,NaT,NaT,5263.86,6113.61,0,564.4,2400,10077.2,31620
is_train,746,NaN,NaN,NaN,NaT,NaT,0,0,0,0,0,0,0
username,746,42,AQU49LCW,127,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
def processing_df(df):
    p_df = df[["user_id", "product_id"]]
    p_df["point"] = 1.0
    p_df = p_df.groupby(["user_id", "product_id"]).point.count().reset_index()
    p_df["minmax_scaled"] = MinMaxScaler().fit_transform(p_df.point.values.astype(float).reshape(-1, 1))
    p_df["standard_scaled"] = StandardScaler().fit_transform(p_df.point.values.astype(float).reshape(-1, 1))
    return p_df

In [19]:
train_data = processing_df(train_df)
train_data

C:\Users\NHAT TRUONG\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,user_id,product_id,point,minmax_scaled,standard_scaled
0,0,42,1,0.00,-0.397880
1,0,287,1,0.00,-0.397880
2,0,288,1,0.00,-0.397880
3,1,327,1,0.00,-0.397880
4,1,328,1,0.00,-0.397880
...,...,...,...,...,...
445,43,272,1,0.00,-0.397880
446,43,275,1,0.00,-0.397880
447,43,276,1,0.00,-0.397880
448,43,277,1,0.00,-0.397880


In [20]:
train_data.describe(include="all").T

,count,mean,std,min,25%,50%,75%,max
user_id,450.0,1.844667e+01,13.290922,0.00000,5.00000,14.00000,30.000000,44.000000
product_id,450.0,2.330044e+02,142.107115,0.00000,112.25000,221.50000,357.750000,491.000000
point,450.0,3.708889e+00,6.815884,1.00000,1.00000,1.00000,3.000000,51.000000
minmax_scaled,450.0,5.417778e-02,0.136318,0.00000,0.00000,0.00000,0.040000,1.000000
standard_scaled,450.0,7.105427e-17,1.001113,-0.39788,-0.39788,-0.39788,-0.104121,6.946091


In [21]:
# Points of users-products 
points = list(train_data.minmax_scaled)
# Get the rows and columns for the above points
u_loc = train_data.user_id.astype(int)
p_loc = train_data.product_id.astype(int)

In [22]:
UserI_sparse = sparse.csr_matrix((points, (u_loc, p_loc)), shape=(len(df2.user_id.unique()), len(df2.product_id.unique())))
ItemU_sparse = sparse.csr_matrix((points, (p_loc, u_loc)), shape=(len(df2.product_id.unique()), len(df2.user_id.unique())))

In [23]:
matrix_size = ItemU_sparse.shape[0]*ItemU_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(ItemU_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.20698924731182

In [24]:
%%time
model_als= implicit.als.AlternatingLeastSquares(factors=256, iterations=100, calculate_training_loss=True)
alpha_val = 100
data_conf = (ItemU_sparse * alpha_val).astype('double')

#Fit the model
model_als.fit(data_conf)


Wall time: 3.31 s


In [25]:
test_data = processing_df(test_df)
total_tp = 0
n_items=50
accuracy = []
for u_id in list(np.sort(test_data.user_id.unique())):
    user_df = test_data[test_data.user_id == u_id]
    user_future_liked_products = np.sort(list(user_df.product_id.unique()))
    total_will_liked_products = len(user_future_liked_products)
    recommended = model_als.recommend(u_id, UserI_sparse, N=n_items, recalculate_user=True, filter_already_liked_items=False)
    TP = 0
    for p, s in recommended:
        if p in user_future_liked_products:
            TP += 1
            total_tp += 1
    if n_items > total_will_liked_products:
        acc = TP/total_will_liked_products
    else:
        acc = TP/n_items        
    accuracy.append(acc*100)
np.mean(accuracy)

C:\Users\NHAT TRUONG\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


49.14778343349771

In [28]:
new_user_items = sparse.csr_matrix(([0.5], ([50], [495])))
recs = model_als.recommend(userid=50, user_items=new_user_items, recalculate_user=True)

ValueError: cannot infer dimensions from zero sized index arrays

In [55]:
df2[df2.user_id == user_id].describe(include="all").T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
user_id,422,NaN,NaN,NaN,NaT,NaT,2,0,2,2,2,2,2
order_id,422,35,XBH011701/023,22,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
time,422,12,2017-01-21 00:00:00,81,2016-06-01,2017-04-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,422,35,Xuất bán T01/023-JETTENBURG - LSX 42,22,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_id,422,NaN,NaN,NaN,NaT,NaT,210.024,146.66,4,83,203,364,479
product_name,422,50,Dinning Sofa Set_Ottoman_F1160,21,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
amount,422,NaN,NaN,NaN,NaT,NaT,30.7607,78.5839,1,6,10,24,960
price,422,NaN,NaN,NaN,NaT,NaT,859.305,1249.28,0,326,519.1,971.6,13500
is_train,422,NaN,NaN,NaN,NaT,NaT,1,0,1,1,1,1,1
username,422,1,AQU49LCW,422,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
user_id = 2

# Use the implicit recommender.
recommended = model_als.recommend(user_id, UserI_sparse, N=491, recalculate_user=True)

product_ids = []
product_names = []
scores = []

# Get artist names from ids
for idx, score in recommended:
    product_ids.append(idx)
    product_names.append(df2.product_name.loc[df2.product_id==idx].iloc[0])
    scores.append(score)

# Create a dataframe of artist names and scores

recommendations = pd.DataFrame({'product': product_ids, 'name': product_names, 'score': scores})


recommendations

,product,name,score
0,45,Cavallino Chair_AL_F310,0.004545
1,183,Steel Wicker Stackable Chair_F413,0.002494
2,19,Bali Chair_F310,0.002339
3,262,Mẫu Gudhjem Chair dây F333 LSXM16034,0.002269
4,209,Henrik Balcony Planter_H274,0.002239
...,...,...,...
452,492,Kính cường lực 5ly Ø490x5,0.000000
453,493,Nút Đế Nhựa f22,0.000000
454,494,Nút nhựa f27.6 (bali),0.000000
455,495,Nút hít kính f30x5 (mm),0.000000


In [42]:
p_loc.max()

491

In [58]:
user_vecs = model_als.user_factors
item_vecs = model_als.item_factors

In [60]:
user_vecs.shape

(45, 256)

In [61]:
item_vecs.shape

(496, 256)

In [ ]:
implicit.als.AlternatingLeastSquares(factors=256, iterations=100, calculate_training_loss=True)

In [63]:
import pickle
import scipy.sparse as sparse

In [64]:
file = open("./model.pkl", "wb")
pickle.dump(model_als, file)

In [65]:
saved_model = pickle.load(open('./model.pkl', 'rb'))

In [66]:
test_data = processing_df(train_test_split_df)

C:\Users\NHAT TRUONG\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [71]:
p_ids = []
p_names = []
scores = []
for idx, score in saved_model.similar_items(45, N=100):
    p_ids.append(idx)
    p_names.append(df2.product_name.loc[df2.product_id==idx].iloc[0])
    scores.append(score)
similar_products = pd.DataFrame({'product': p_ids, 'name': p_names, 'score': scores})
similar_products.head(20)

,product,name,score
0,45,Cavallino Chair_AL_F310,0.161968
1,312,Mẫu Rect Table_F310,0.029363
2,262,Mẫu Gudhjem Chair dây F333 LSXM16034,0.029225
3,76,Gudhjem Chair_F333,0.029164
4,270,Mẫu 3 seater sofa S70_F333 LSXM16034,0.029164
5,460,Rect Table_H274,0.029136
6,142,Nệm ngồi Scout single lounge,0.029132
7,210,Long Planter_H274,0.029079
8,349,Arsenal Small Stool_F332,0.029064
9,104,Mansan Chair F343,0.029053


In [67]:
p_ids = [int(idx) for idx, score in saved_model.similar_items(45, N=100)]45

[(45, 0.16196753),
 (312, 0.029362889),
 (262, 0.02922479),
 (76, 0.029164273),
 (270, 0.029163772),
 (460, 0.029135544),
 (142, 0.02913214),
 (210, 0.029078884),
 (349, 0.029063663),
 (104, 0.029053155),
 (209, 0.029044691),
 (143, 0.029039817),
 (290, 0.029032093),
 (170, 0.029016962),
 (269, 0.029008394),
 (216, 0.028985782),
 (293, 0.028976774),
 (347, 0.028963208),
 (244, 0.028961131),
 (218, 0.028945243),
 (285, 0.028933128),
 (259, 0.028931517),
 (432, 0.028931072),
 (182, 0.028922057),
 (350, 0.02888029),
 (277, 0.028860364),
 (82, 0.02885649),
 (86, 0.028825318),
 (93, 0.028824933),
 (219, 0.028824236),
 (33, 0.028822612),
 (27, 0.02881976),
 (329, 0.028818324),
 (287, 0.02881338),
 (117, 0.028808694),
 (283, 0.028808651),
 (18, 0.028805982),
 (302, 0.028805884),
 (428, 0.028792305),
 (444, 0.028774576),
 (53, 0.028774356),
 (477, 0.028767776),
 (42, 0.028765295),
 (208, 0.028756086),
 (330, 0.028750973),
 (75, 0.028746223),
 (440, 0.02873631),
 (352, 0.028736236),
 (403, 0.02

In [102]:
from google_images_download import google_images_download  
  
# creating object 
response = google_images_download.googleimagesdownload()  
  

def downloadimages(query): 
    # keywords is the search query 
    # format is the image file format 
    # limit is the number of images to be downloaded 
    # print urs is to print the image file url 
    # size is the image size which can 
    # be specified manually ("large, medium, icon") 
    # aspect ratio denotes the height width ratio 
    # of images to download. ("tall, square, wide, panoramic") 
    arguments = {"keywords": query, 
                 "format": "jpg", 
                 "limit":4, 
#                  "print_urls":True, 
                 "size": "medium", 
                 "aspect_ratio": "tall",
                 "type": "photo",
                 "output_directory": "./imgs",
                 "no_directory": True,
#                  "save_source": "imgsrc",
                 "silent_mode": True,
#                  "specific_site": "amazon.com",
                 "no_numbering": True}
    try: 
        return response.download(arguments) 
      
    # Handling File NotFound Error     
    except FileNotFoundError:  
        arguments = {"keywords": query, 
                     "format": "jpg", 
                     "limit":4, 
#                      "print_urls":True,  
                     "size": "medium",
                     "type": "photo",
                     "output_directory": "./imgs",
                     "no_directory": True,
#                      "save_source": "imgsrc",
                     "silent_mode": True,
#                      "specific_site": "amazon.com",
                     "no_numbering": True} 
                       
        # Providing arguments for the searched query 
        try: 
            # Downloading the photos based 
            # on the given arguments 
            return response.download(arguments)  
        except: 
            pass
  
temp_query = "furniture Diana Lounger F268"
path = downloadimages(temp_query)


In [103]:
path[0].get(temp_query)

['F:\\workspace\\RecSys_ecommerce\\recsys\\imgs\\page_1_thumb_large.jpg',
 'F:\\workspace\\RecSys_ecommerce\\recsys\\imgs\\1_8ff87a15-bcd6-4cfb-b843-45cf84ff0fbe_800x.jpg',
 'F:\\workspace\\RecSys_ecommerce\\recsys\\imgs\\1_c28872b0-ecd9-499d-8c18-e7020957c24b_800x.jpg',
 'F:\\workspace\\RecSys_ecommerce\\recsys\\imgs\\e80e62f7-9061-453a-8c30-048f47b2c3fc-image.jpg']

In [54]:
path[0].get(temp_query)[0].split("\\")[-1]

'2c834cd18803b483b587443976adea7d.jpg'

In [56]:
path[0].get(temp_query)[0]

'F:\\workspace\\RecSys_ecommerce\\recsys\\imgs\\2c834cd18803b483b587443976adea7d.jpg'

In [55]:
import glob
import shutil
import os

dst_dir = "./downloads"
for jpgfile in glob.iglob(path[0].get(temp_query)[0]):
    shutil.copy(jpgfile, dst_dir)

In [41]:
path[0].items()

dict_items([('The smartphone could be fuelled by a 3 700mAh battery.', ['https://ivslabreviews.files.wordpress.com/2019/06/9-3.jpg', 'https://appuals.com/wp-content/uploads/2019/04/1-2.jpg', 'https://www.xiaomitoday.com/wp-content/uploads/2019/09/Smok-Mico-Starter-Kit-1.jpg', 'https://images-na.ssl-images-amazon.com/images/I/812uVILgNCL._SX679_.jpg'])])

In [6]:
from datetime import datetime

In [7]:
df = pd.read_csv("./data/events.csv", encoding="utf-8")

In [8]:
df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [9]:
df["time"] = df.timestamp.apply(lambda x: datetime.utcfromtimestamp(x//1000.0).strftime('%Y-%m-%d %H:%M:%S'))
df = df.sort_values(by=["time"])
df.drop(columns=["transactionid", "timestamp"], inplace=True)

In [10]:
df.visitorid = df.visitorid.astype(str)
df.itemid = df.itemid.astype(str)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2756101 entries, 1462974 to 1459312
Data columns (total 4 columns):
visitorid    object
event        object
itemid       object
time         object
dtypes: object(4)
memory usage: 105.1+ MB


In [13]:
df.describe(include="all").T

,count,unique,top,freq
visitorid,2756101,1407580,1150086,7757
event,2756101,3,view,2664312
itemid,2756101,235061,187946,3412
time,2756101,2357232,2015-05-14 16:40:13,28


In [14]:
df.head(3)

,visitorid,event,itemid,time
1462974,693516,addtocart,297662,2015-05-03 03:00:04
1464806,829044,view,60987,2015-05-03 03:00:11
1463000,652699,view,252860,2015-05-03 03:00:13


### Summary
- user
    - Total: 1407580
    - Most freq buyer: 1150086 (7757 times)
    - 
- product
    - Total: 235061
    - Most popular product: 187946 (3412 times)

In [15]:
df2.head()

,user_id,order_id,time,description,product_id,product_name,amount,price
0,BDS49BDS,XBH011609/006,2016-09-05,Xuất bán T09/06- DYKEH ngày ETD 5/9/2016,LTSCORMH374SCA1,Corner Sofa Set_Middle_H374,68.0,1812.88
1,BDS49BDS,XBH011609/005,2016-09-05,Xuất bán T09/05- DYKEH ngày ETD 5/9/2016,LTSCORMH374SCA1,Corner Sofa Set_Middle_H374,68.0,1812.88
2,BDS49BDS,XBH011609/005,2016-09-05,Xuất bán T09/05- DYKEH ngày ETD 5/9/2016,LTSCORCH374SCA1,Corner Sofa Set_Corner_H374,34.0,1813.22
3,BDS49BDS,XBH011609/005,2016-09-05,Xuất bán T09/05- DYKEH ngày ETD 5/9/2016,LTSCOR1H374SCA1,Corner Sofa Set_ArmChair_H374,34.0,1813.22
4,BDS49BDS,XBH011609/004,2016-09-05,Xuất bán T09/04- DYKEH ngày ETD 5/9/2016,LTTCORVH374SCA1,Corner Sofa Set_Table_H374,34.0,1814.24


In [16]:
df2["event"] = "transaction"

In [17]:
df2.head(2)

,user_id,order_id,time,description,product_id,product_name,amount,price,event
0,BDS49BDS,XBH011609/006,2016-09-05,Xuất bán T09/06- DYKEH ngày ETD 5/9/2016,LTSCORMH374SCA1,Corner Sofa Set_Middle_H374,68.0,1812.88,transaction
1,BDS49BDS,XBH011609/005,2016-09-05,Xuất bán T09/05- DYKEH ngày ETD 5/9/2016,LTSCORMH374SCA1,Corner Sofa Set_Middle_H374,68.0,1812.88,transaction


In [20]:
import random
df["product_name"] = df.itemid.apply(lambda x: "item noname #{}".format(x))
listprice = df2.price.unique()
df["price"] = df.itemid.apply(lambda x: random.choice(listprice))
df["amount"] = 1.0
df.head(2)

,visitorid,event,itemid,time,product_name,price,amount
1462974,693516,addtocart,297662,2015-05-03 03:00:04,item noname #297662,82.0,1.0
1464806,829044,view,60987,2015-05-03 03:00:11,item noname #60987,10266.0,1.0


In [21]:
df2 = df2[['time', 'event', 'user_id', 'product_id', "product_name", "amount", "price"]]
df = df[['time', 'event', 'visitorid', 'itemid', "product_name", "amount", "price"]]
new_columns = ['time', 'event', 'user_id', 'product_id', "product_name", "amount", "price"]
df.columns = new_columns
df2.columns = new_columns

In [22]:
df.append(df2, ignore_index=True)

,time,event,user_id,product_id,product_name,amount,price
0,2015-05-03 03:00:04,addtocart,693516,297662,item noname #297662,1.0,82.0
1,2015-05-03 03:00:11,view,829044,60987,item noname #60987,1.0,10266.0
2,2015-05-03 03:00:13,view,652699,252860,item noname #252860,1.0,4920.0
3,2015-05-03 03:00:24,view,1125936,33661,item noname #33661,1.0,9015.5
4,2015-05-03 03:00:26,view,693516,297662,item noname #297662,1.0,2235.6
...,...,...,...,...,...,...,...
2758511,2017-01-10,transaction,MET852MGB,LTCPRI1F310SK1,Prisca Chair_F310,540.0,8262.0
2758512,2017-01-10,transaction,MET852MGB,LTCBIS1F310SP1,Bistro Chair_F310,336.0,3528.0
2758513,2017-01-10,transaction,LIK45JYS,LTCNEW1F310SCA1,Idskov Lounge Set_Armchair_F310,160.0,1696.0
2758514,2017-01-10,transaction,LIK45JYS,LTCNEW2F310SCA1,Idskov Lounge Set_Bench_F310,80.0,3253.6


In [23]:
newdf = df.append(df2, ignore_index=True)

In [24]:
newdf

,time,event,user_id,product_id,product_name,amount,price
0,2015-05-03 03:00:04,addtocart,693516,297662,item noname #297662,1.0,82.0
1,2015-05-03 03:00:11,view,829044,60987,item noname #60987,1.0,10266.0
2,2015-05-03 03:00:13,view,652699,252860,item noname #252860,1.0,4920.0
3,2015-05-03 03:00:24,view,1125936,33661,item noname #33661,1.0,9015.5
4,2015-05-03 03:00:26,view,693516,297662,item noname #297662,1.0,2235.6
...,...,...,...,...,...,...,...
2758511,2017-01-10,transaction,MET852MGB,LTCPRI1F310SK1,Prisca Chair_F310,540.0,8262.0
2758512,2017-01-10,transaction,MET852MGB,LTCBIS1F310SP1,Bistro Chair_F310,336.0,3528.0
2758513,2017-01-10,transaction,LIK45JYS,LTCNEW1F310SCA1,Idskov Lounge Set_Armchair_F310,160.0,1696.0
2758514,2017-01-10,transaction,LIK45JYS,LTCNEW2F310SCA1,Idskov Lounge Set_Bench_F310,80.0,3253.6


In [29]:
newdf.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
time,2758516,2357456,2017-01-21,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN
event,2758516,3,view,2664312,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_id,2.75852e+06,NaN,NaN,NaN,706606,406531,0,355922,707408,1.0596e+06,1.40762e+06
product_id,2.75852e+06,NaN,NaN,NaN,117964,67649.9,0,60099,117317,176517,235556
product_name,2758516,235556,item noname #187946,3412,NaN,NaN,NaN,NaN,NaN,NaN,NaN
amount,2.75852e+06,NaN,NaN,NaN,1.3342,20.2253,1,1,1,1,5040
price,2.75852e+06,NaN,NaN,NaN,3601.19,4748.36,0,552,1870,4508,31620


In [27]:
newdf["user_id"] = newdf["user_id"].astype("category").cat.codes
newdf["product_id"] = newdf["product_id"].astype("category").cat.codes

In [28]:
newdf

,time,event,user_id,product_id,product_name,amount,price
0,2015-05-03 03:00:04,addtocart,1067045,110526,item noname #297662,1.0,82.0
1,2015-05-03 03:00:11,view,1217632,213271,item noname #60987,1.0,10266.0
2,2015-05-03 03:00:13,view,1021692,85424,item noname #252860,1.0,4920.0
3,2015-05-03 03:00:24,view,139933,132332,item noname #33661,1.0,9015.5
4,2015-05-03 03:00:26,view,1067045,110526,item noname #297662,1.0,2235.6
...,...,...,...,...,...,...,...
2758511,2017-01-10,transaction,1407607,235185,Prisca Chair_F310,540.0,8262.0
2758512,2017-01-10,transaction,1407607,235091,Bistro Chair_F310,336.0,3528.0
2758513,2017-01-10,transaction,1407601,235173,Idskov Lounge Set_Armchair_F310,160.0,1696.0
2758514,2017-01-10,transaction,1407601,235174,Idskov Lounge Set_Bench_F310,80.0,3253.6


In [31]:
newdf.to_csv("data/fullevent_db.csv", sep=";", index=False)